# Neural Net Training Workspace
*Work produced by Stephen Toner, Fall 2022*

In [6]:
import torch
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.model_selection import train_test_split
import Utils as U

params = {'legend.fontsize': 'x-large',
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
       'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [4]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [5]:

data = pd.read_csv("timeseries_readyformodel.csv")

y = data['died']
X = data.drop(columns = ['died', 'patientunitstayid']).to_numpy().astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    # stratify=X[['hospitalregion',
                                                    #             'teachingstatus']],
                                                    random_state=607)


In [13]:
# scaler = StandardScaler()
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

X_train_scaled = X_train
X_test_scaled = X_test

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
net = models.Sequential()
net.add(layers.Dense(16, activation = 'tanh', input_shape = X_train_scaled.shape))
net.add(layers.Dense(32, activation = 'tanh'))
net.add(layers.Dense(64, activation = 'tanh')) # Up to 82%
net.add(layers.Dense(128, activation = 'tanh')) # Up to 82.5%
net.add(layers.Dense(256, activation = 'tanh')) # 
net.add(layers.Dense(512, activation = 'tanh')) # 
# net.add(layers.Conv1D(256, kernel_size = 2))# 
net.add(layers.Dense(256, activation = 'relu')) # 85% for flat 256, 3 tanh layers; down to 84% when increasing powers of 2
# net.add(layers.Dense(32, activation = 'relu'))
# net.add(layers.Dense(16, activation = 'relu'))
# net.add(layers.Dense(8, activation = 'relu'))

# Output Layer
net.add(layers.Dense(2, activation = 'sigmoid'))


In [ ]:
net.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['Accuracy'])
net.fit(X_train_scaled, y_train_cat, epochs = 1000, batch_size = 200)#, verbose = False)

In [ ]:
test_accuracy = net.evaluate(X_test_scaled, y_test_cat)[1]

In [ ]:
print(f'Test Accuracy: {test_accuracy}')
eval_metrics = U.evaluation_metrics(X_train_scaled, y_train_cat, X_test_scaled, y_test_cat)

In [ ]:
y_pred_prob = net.predict_prob()
results = U.compute_metrics(y_pred_prob, y_test)

In [11]:
model = load_model('LSTM_model/')

In [15]:
X_tst_ts = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [16]:
model.evaluate(X_tst_ts, y_test_cat)

1569/1569 [==============================] - 1s 589us/step - loss: 1.1583 - auc: 0.8726 - recall: 0.8726 - precision: 0.8726


[1.1583129167556763,
 0.8725689649581909,
 0.8725689649581909,
 0.8725689649581909]